# CVRP Notebook
This notebook is intended to run the top level code for the CVRP

In [1]:
import networkx as nx

from model import VRP, create_constraints, output_variables
from pricer import VRPPricer
from parse import parse

from matplotlib import pyplot as plt

In [27]:
# Test instance E-n22-k4 provided by parser
# G = parse("Instances/E/E-n22-k4.vrp")
# G = parse("Instances/E/E-n23-k3.vrp")
# G = parse("Instances/E/E-n30-k3.vrp")
# G = parse("Instances/E/E-n33-k4.vrp")
# G = parse("Instances/E/E-n51-k5.vrp")
# G = parse("Instances/E/E-n76-k14.vrp")
# G = parse("Instances/E/E-n101-k14.vrp")
# G = parse("Instances/Uchoa/X-n101-k25.vrp")
# G.graph['min_trucks'] = 26
# G = parse("Instances/Uchoa/X-n106-k14.vrp")
# G.graph['min_trucks'] = 14
# G = parse("Instances/Uchoa/X-n110-k13.vrp")
# G.graph['min_trucks'] = 13
G = parse("Instances/Uchoa/X-n115-k10.vrp")
G.graph['min_trucks'] = 10
# G = parse("Instances/Uchoa/X-n120-k6.vrp")
# G.graph['min_trucks'] = 6
# G = parse("Instances/Uchoa/X-n125-k30.vrp")
# G.graph['min_trucks'] = 30
# G = parse("Instances/Uchoa/X-n129-k18.vrp")
# G.graph['min_trucks'] = 18
# G = parse("Instances/Uchoa/X-n200-k36.vrp")
# G.graph['min_trucks'] = 36
# G = parse("Instances/Uchoa/X-n228-k23.vrp")
# G.graph['min_trucks'] = 23
# G = parse("Instances/Uchoa/X-n251-k28.vrp")
# G.graph['min_trucks'] = 28
# G = parse("Instances/Uchoa/X-n256-k16.vrp")
# G.graph['min_trucks'] = 16
# G = parse("Instances/Uchoa/X-n289-k60.vrp")
# G.graph['min_trucks'] = 61
# G = parse("Instances/Uchoa/X-n502-k39.vrp")
# G.graph['min_trucks'] = 39 #TODO: Ist dieses K auch das K, einer Optimallösung?

PARSE: There is no minimum number of trucks.


In [ ]:
model = VRP(G)

# Create pricer
pricer = VRPPricer()
pricer.init_data(G)
pricer.data['methods'] = ["SPPRC"]
pricer.data['max_vars']= 10000
pricer.data['time_limit'] = 60

model.includePricer(pricer, "pricer","does pricing")

create_constraints(model,G,heuristic_time=1)

model.optimize()
model.hideOutput(quiet=False)

HYGESE: Found Solution with value 12747.0
PRICER_PY: The maximal path length is 171
PY PRICING: The neighborhood has been fixed to 8 neighbors.
PRICER_C: Graph data successfully copied to C.
presolving:
presolving (1 rounds: 1 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 124 variables (0 bin, 0 int, 0 impl, 124 cont) and 115 constraints
    115 constraints of type <linear>
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
* 0.0s|     1 |     0 |    22 |     - |    LP  |   0 | 124 | 115 | 115 |   0 |  0 |   0 |   0 |      --      | 1.274700e+04 |    Inf | unknown
 28.8s|     1 |     0 |    25 |     - |    30M |   0 |  10k| 115 | 115 |   0 |  0 |   0 |   0 |      --      | 1.274700e+04 |    Inf | unknown
 

14704.875062949766


In [25]:
plt.figure(figsize=(18,6),dpi=300)
xmax, max_lb = 0, 0
for method in pricer.data['methods']:
    upper_bounds = [x for x,y in pricer.data['bounds'][method]]
    lower_bounds = [y for x,y in pricer.data['bounds'][method]]
    plt.plot(upper_bounds, label=f'{method} UB')
    plt.plot(lower_bounds, label=f'{method} LB')
    xmax = max(upper_bounds[0],xmax)
    if max(lower_bounds) > upper_bounds[-1]:
        print(f"The maximum lower bound for {method} is larger than the final bound")
    max_lb = max(max(lower_bounds),max_lb)

T = nx.minimum_spanning_tree(G)
mst = T.size(weight='weight') + min(G.edges().data('weight'),key=lambda edge: edge[2])[2]
mst = [mst for i in range(len(upper_bounds))]
plt.plot(mst, label="MST Bound")

max_lb = [max_lb for i in range(len(upper_bounds))]
plt.plot(max_lb, label="Maximal lower bound.")

solval = model.getObjVal()
sol = [solval for i in range(len(upper_bounds))]
plt.plot(sol,label="Solution value")

plt.ylim((mst[0]*0.9,xmax*1.1))

print(max_lb[0])

plt.legend(loc='upper right')
# plt.savefig("plots/X-n502-k39.svg")
plt.show()

In [26]:
output_variables(model, pricer)

The solution contains the following paths: 
Only paths with associated value larger than 1e-06 are analysed.
0.03578629032258138 * x80412: (0, 71, 2, 99, 65, 59, 17, 31, 28, 0)
0.04166666666666663 * x91253: (0, 99, 36, 64, 61, 22, 83, 76, 55, 0)
0.2768817204301074 * x93950: (0, 48, 7, 37, 94, 81, 44, 70, 23, 0)
0.16935483870967777 * x94273: (0, 48, 7, 37, 94, 81, 44, 70, 13, 0)
0.1922043010752685 * x94580: (0, 48, 82, 7, 37, 94, 81, 44, 70, 0)
0.08467741935483886 * x98254: (0, 13, 57, 82, 107, 7, 35, 73, 47, 0)
0.08467741935483891 * x103553: (0, 54, 96, 88, 75, 21, 72, 63, 10, 100, 0)
0.08467741935483886 * x103850: (0, 70, 44, 81, 94, 37, 35, 73, 47, 40, 0)
0.04166666666666663 * x103909: (0, 99, 64, 104, 60, 61, 83, 76, 55, 56, 0)
0.6976744186046512 * x104302: (0, 91, 5, 46, 38, 108, 3, 14, 0)
The following path is non elementary
0.10077519379844962 * x104327: (0, 14, 3, 108, 46, 5, 91, 3, 0)
The following path is non elementary
0.2540322580645167 * x104418: (0, 62, 100, 10, 86, 47, 40